### **6.2 - Make your choices with Branching**

"Branching" es un concepto bastante fácil de entender, pero extremadamente potente. En efecto, con branching puedes elegir una tarea u otra según una condición. Si tu condición es true, entonces vas a devolver un task ID o múltiples task IDs correspondientes a las tareas que quieres ejecutar a continuación. Y en este video te voy a mostrar el Operador BranchPython, pero no es el único, también existe el Operador BranchSql que te permite ejecutar una tarea u otra de acuerdo a un valor en una tabla SQL. También tienes el Operador BranchDatetime que te permite ejecutar una tarea u otra de acuerdo a un marco de tiempo (timeframe) dado, por lo que puedes tener diferentes schedules en tu data pipeline. Por último, el operador BranchDayOfWeek permite ejecutar una tarea u otra en función del día actual. Así que no olvides que el Operador BranchPython es extremadamente popular, pero también hay otros Operadores Branch. Dicho esto, vamos a sumergirnos en el código. 

En tu data pipeline, imaginemos que quieres extraer datos para Snowflake sólo si el día actual es lunes. Luego, para Netflix, quieres extraer los datos sólo si el día actual es miércoles y para Astronomer quieres extraer los datos sólo si el día actual es viernes. ¿Cómo se puede hacer eso, ya que el data pipeline está programado (scheduled) @daily? 

<center><img src="https://i.postimg.cc/yN6xmfK4/a1328.png"></center>

Bueno en ese caso una cosa que puedes hacer es usar el Operador BranchPython. Así que vamos a hacerlo. Aquí solo necesitas importar el Operador BranchPython:

<center><img src="https://i.postimg.cc/5tP0nw9H/a1329.png"></center>

y luego puedes copiar ese operador y crear una nueva tarea. Así que vamos a crear esa nueva tarea. El último argumento que necesitas especificar es "python_callable" que espera la función encargada de devolver los task IDs de la siguiente tarea a ejecutar basada en tu condición. Así que vamos a crear esa función llamada "choosing_partner_based_on_day":

<center><img src="https://i.postimg.cc/5Nw2ZcNW/a1330.png"></center>

Y aquí puedes crear la función "_choosing_partner_based_on_day( )" y ahora haces lo que quieras. Así que como tu objetivo es ejecutar "extract_partner_netflix" o "extract_partner_astronomer" o "extract_partner_snowflake" según el día actual, necesitas usar la 'execution date'. Así que aquí como argumento, puedes pasar 'execution date' y en base a ella, obtener el día:

<center><img src="https://i.postimg.cc/5Nct79Jm/a1331.png"></center>

Necesitas especificar el task ID de la siguiente tarea que quieres ejecutar de acuerdo a tu condición. Ahora, una cosa que tienes que saber es que siempre debes "retornar (return)" algo. Con el Operador BranchPython si no devuelves un task ID, entonces terminarás con un error. Así que aquí vas a tener un problema para los demás días. Entonces, ¿cómo puedes resolver esto? Bueno, hay diferentes maneras. En primer lugar, puedes modificar el "scheduling interval" de tu "data pipeline", de forma que sólo se active los lunes, miércoles y viernes. O puedes añadir otra tarea, por ejemplo, llamémosla "stop":

<center><img src="https://i.postimg.cc/NjDfj1HN/a1332.png"></center>

y entonces básicamente vas a ejecutar esa tarea si el día actual no es viernes, miércoles o lunes:

<center><img src="https://i.postimg.cc/BbFnSkTD/a1333.png"></center>

Además, otra solución podría ser usar el Operador ShortCircuit como la primera tarea de tu data pipeline y lo mismo, tendrás una condición para verificar si el día actual es viernes, miércoles o lunes. Si no es así, entonces saltará todas las demás tareas de tu data pipeline. Así que hay diferentes maneras, pero vamos a seguir con eso. Es la más sencilla. Ahora, tenemos que definir las dependencias:

<center><img src="https://i.postimg.cc/g2m2pMnX/a1334.png"></center>

Guarde el archivo y vamos a ver lo que se obtiene en la interfaz de usuario de Airflow. Haz clic en "my_dag", ve a la Graph view y obtendrás el siguiente data pipeline:

<center><img src="https://i.postimg.cc/hj6Gqcdc/a1335.png"></center>

Aquí puedes ver el Operador BranchPython, que elegirá entre "extract_partner_astronomer", "extract_partner_netflix" o "extract_partner_snowflake" según el día actual. Y si el día actual no es ni lunes, ni miércoles, ni viernes, entonces se activará la tarea stop. Veamos lo que se obtiene si se activa el toogle del dag, se refresca la página, se espera un poco y como se puede ver en función del día actual, "extract_partner_netflix" se dispara y las otras tareas se saltan. Eso es lo que puedes ver por los colores de los bordes de las tareas:

<center><img src="https://i.postimg.cc/HW8x9cqx/a1336.png"></center>

Ahora, me gustaría mostrarte algo más. ¿Qué pasa si tienes una tarea al final de tu data pipeline, como "store" por ejemplo, y bien, algunos de los padres se omiten para esa tarea? ¿Qué pasaría? Porque es realmente un problema común con el Operador BranchPython. Así que permítanme mostrarles exactamente el problema con el que terminarán. Digamos que tienes una tarea para ejecutar al final de tu data pipeline, que es "storing":

<center><img src="https://i.postimg.cc/yNN6gkh7/a1337.png"></center>

y luego quieres poner esa tarea como la última tarea a ejecutar, así que digamos que justo después de "process_tasks" quieres ejecutar "storing":

<center><img src="https://i.postimg.cc/MGr6yqtV/a1338.png"></center>

Guarde el archivo y vuelva a la interfaz de Airflow. Ya tiene la tarea "storing":

<center><img src="https://i.postimg.cc/gJXYsQB5/a1339.png"></center>

Volvamos a activar el toggle del DAG para ver que se obtiene. Actualiza la página, espera un poco y aquí hay un problema. De hecho, puedes ver que "storing" se salta incluso si "extract_partner_netflix" se ha ejecutado con éxito:

<center><img src="https://i.postimg.cc/rmBqPjBp/a1340.png"></center>

¿Por qué? porque uno de los padres, en realidad, múltiples padres de la tarea "storing" se han omitido. Eso es lo que puedes ver con "process_tasks" y "process_tasks__2". Entonces, ¿cómo puedes asegurarte de que tu tarea se dispara si uno de los padres ha tenido éxito, incluso si los otros padres han sido omitidos? Bueno, aquí es donde necesitas utilizar un nuevo concepto. El concepto de "Trigger rules" y lo vas a ver justo después del video, pero para resolver este problema, déjame mostrarte ahora mismo cómo hacerlo. Así que, si regresas a tu editor de código, una cosa que tienes que añadir a la tarea "storing" es:

<center><img src="https://i.postimg.cc/gj3GFRpH/a1341.png"></center>

Si guardas el archivo y vuelves a intentarlo, haz clic en la tarea "start" y luego en "Clear":

<center><img src="https://i.postimg.cc/3JHKjLjS/a1342.png"></center>
<center><img src="https://i.postimg.cc/MZB6QkBf/a1343.png"></center>

Esperemos un poco y como puedes ver, esta vez, la tarea "storing" se ha ejecutado con éxito.

<center><img src="https://i.postimg.cc/xTy96Kt0/a1344.png"></center>

Así es como se puede resolver este problema muy común con el BranchPython Operator mediante el uso de "Trigger rules". Así que el Operador BranchPython es realmente poderoso porque te permite crear workflows mucho más complejos que sin él. No olvides que puede devolver múltiples task IDs por lo que puedes ejecutar múltiples tareas al mismo tiempo de acuerdo a una condición. Además, existen otros operadores Branch. No te centres siempre sólo en el Operador BranchPython, existe el Operador BranchSql, el Operador BranchDatetime y el Operador BranchDayOfTheWeek. Pueden ser realmente útiles. Dicho esto, vamos a sumergirnos en las "Trigger rules".